In [7]:
import xarray as xr
import rasterio
from rasterio.transform import from_origin
import numpy as np
import os
import glob

# --- CONFIGURATION ---
INPUT_NC_FOLDER = 'input/nc'
OUTPUT_TIF_FOLDER = 'input/tif'
MAX_FILES_TO_KEEP = 12
LAT_MIN, LAT_MAX = -5.0, 5.0
LON_MIN, LON_MAX = 108.0, 119.0
# --- END OF CONFIGURATION ---


def cleanup_old_files(directory, max_files):
    """
    Ensures that the number of .tif files in a directory does not exceed a limit.
    The oldest files (by modification time) are deleted first.
    """
    try:
        files = glob.glob(os.path.join(directory, '*.tif'))
        files = [f for f in files if os.path.isfile(f)]

        if len(files) <= max_files:
            print(f"Cleanup not needed. Found {len(files)} files (limit is {max_files}).")
            return

        files.sort(key=os.path.getmtime)
        num_to_delete = len(files) - max_files
        files_to_delete = files[:num_to_delete]
        
        print(f"Found {len(files)} files. Deleting the {num_to_delete} oldest files.")
        for f_path in files_to_delete:
            try:
                print(f" -> Deleting: {os.path.basename(f_path)}")
                os.remove(f_path)
            except OSError as e:
                print(f"    Error deleting file {f_path}: {e}")

    except Exception as e:
        print(f"\n An error occurred during file cleanup: {e}")


def main():
    os.makedirs(OUTPUT_TIF_FOLDER, exist_ok=True)
    nc_files = glob.glob(os.path.join(INPUT_NC_FOLDER, '*.nc'))
    for input_file_path in nc_files:
        try:
            filename_base = os.path.basename(input_file_path)
            timestamp_part = filename_base.split('_s')[1].split('_e')[0]
            timestamp = timestamp_part[:12]
            output_filename = f'RRQPE_{timestamp}.tif'
            output_path_tif = os.path.join(OUTPUT_TIF_FOLDER, output_filename)
            if os.path.exists(output_path_tif):
                print(f"Skipping: '{output_filename}' already exists.")
                continue
            with xr.open_dataset(input_file_path) as ds:
                rrqpe = ds['RRQPE']
                if 'scale_factor' in rrqpe.attrs:
                    rrqpe_scaled = rrqpe #masih ga tau apakah harus di kali dengan scale factor apa nggak
                else:
                    rrqpe_scaled = rrqpe
                if 'latitude' in ds.variables and 'longitude' in ds.variables:
                    latitudes = ds['latitude'].values
                    longitudes = ds['longitude'].values
                    lat_dim_name = 'latitude'
                    lon_dim_name = 'longitude'
                else:
                    latitudes = np.arange(ds.attrs['geospatial_lat_max'], ds.attrs['geospatial_lat_min'] - ds.attrs['geospatial_lat_resolution'], -ds.attrs['geospatial_lat_resolution'])
                    longitudes = np.arange(ds.attrs['geospatial_lon_min'], ds.attrs['geospatial_lon_max'] + ds.attrs['geospatial_lon_resolution'], ds.attrs['geospatial_lon_resolution'])
                    
                    if 'Rows' in rrqpe.dims and 'Columns' in rrqpe.dims:
                        lat_dim_name = 'Rows'
                        lon_dim_name = 'Columns'
                    else:
                        print(f"   ERROR: Could not find recognizable dimension names in {rrqpe.dims}. Skipping.")
                        continue

                lat_indices = np.where((latitudes >= LAT_MIN) & (latitudes <= LAT_MAX))[0]
                lon_indices = np.where((longitudes >= LON_MIN) & (longitudes <= LON_MAX))[0]
                
                if len(lat_indices) == 0 or len(lon_indices) == 0:
                    print("   Warning: The file's coordinates do not overlap with the defined clipping bounds. Skipping file.")
                    continue
                
                indexer = {lat_dim_name: lat_indices, lon_dim_name: lon_indices}
                rrqpe_clipped = rrqpe_scaled.isel(**indexer)
                latitudes_clipped = latitudes[lat_indices]
                longitudes_clipped = longitudes[lon_indices]
                
                # --- 4. Prepare for GeoTIFF creation ---
                lat_resolution = abs(latitudes_clipped[1] - latitudes_clipped[0]) if len(latitudes_clipped) > 1 else ds.attrs.get('geospatial_lat_resolution', 0.01)
                lon_resolution = abs(longitudes_clipped[1] - longitudes_clipped[0]) if len(longitudes_clipped) > 1 else ds.attrs.get('geospatial_lon_resolution', 0.01)
                transform = from_origin(longitudes_clipped.min(), latitudes_clipped.max(), lon_resolution, lat_resolution)
                rrqpe_data = rrqpe_clipped.values
                if '_FillValue' in rrqpe.attrs:
                    rrqpe_data[rrqpe_data == rrqpe.attrs['_FillValue']] = np.nan

                # --- 5. Save the GeoTIFF file ---
                with rasterio.open(
                    output_path_tif,
                    'w',
                    driver='GTiff',
                    height=rrqpe_data.shape[0],
                    width=rrqpe_data.shape[1],
                    count=1,
                    dtype=str(rrqpe_data.dtype),
                    crs='EPSG:4326',
                    transform=transform,
                    nodata=np.nan
                ) as dst:
                    dst.write(rrqpe_data, 1)
                
                print(f" Success! Saved to: {output_filename}")

        except (IndexError, KeyError) as e:
            print(f" -> ERROR: Could not parse filename or find expected variable in '{filename_base}'. Details: {e}")
        except Exception as e:
            print(f" -> An unexpected ERROR occurred while processing '{filename_base}': {e}")
    
    # --- limit to 12 ---
    cleanup_old_files(OUTPUT_TIF_FOLDER, MAX_FILES_TO_KEEP)
    
    print("\n--- Process Finished ---")

if __name__ == "__main__":
    main()


    

 Success! Saved to: RRQPE_202509290750.tif
 Success! Saved to: RRQPE_202509290800.tif
 Success! Saved to: RRQPE_202509290810.tif
 Success! Saved to: RRQPE_202509290820.tif
 Success! Saved to: RRQPE_202509290830.tif
 Success! Saved to: RRQPE_202509290840.tif
 Success! Saved to: RRQPE_202509290850.tif
 Success! Saved to: RRQPE_202509290900.tif
 Success! Saved to: RRQPE_202509290720.tif
 Success! Saved to: RRQPE_202509290730.tif
 Success! Saved to: RRQPE_202509290740.tif
 Success! Saved to: RRQPE_202509290910.tif
Cleanup not needed. Found 12 files (limit is 12).

--- Process Finished ---


In [3]:
import xarray as xr
import rasterio
import os
import glob
import numpy as np

# --- SETTINGS ---
INPUT_FOLDER = 'input/nc'
OUTPUT_FOLDER = 'input/tif'
MAX_FILES_TO_KEEP = 12
LAT_MIN, LAT_MAX = -5.0, 5.0
LON_MIN, LON_MAX = 108.0, 119.0

# --- PROCESS ---
print("--- Starting Conversion Process ---")
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
nc_files = glob.glob(os.path.join(INPUT_FOLDER, '*.nc'))

if not nc_files:
    print(f"No .nc files found in '{INPUT_FOLDER}'. Nothing to do.")
else:
    print(f"Found {len(nc_files)} files to process.\n")

    for nc_file_path in nc_files:
        try:
            base_filename = os.path.basename(nc_file_path)
            
            timestamp = base_filename.split('_s')[1].split('_e')[0]            
            output_filename = f'Rainfall_{timestamp}.tif'
            output_path = os.path.join(OUTPUT_FOLDER, output_filename)

            if os.path.exists(output_path):
                print(f"Skipping: '{output_filename}' already exists.")
                continue

            print(f"Processing: '{base_filename}'")

            with xr.open_dataset(nc_file_path) as ds:
                data_variable = ds['RRQPE']

                cropped_data = data_variable.sel(
                    latitude=slice(LAT_MAX, LAT_MIN),
                    longitude=slice(LON_MIN, LON_MAX)
                )
                
                cropped_data.rio.to_raster(output_path)
            
            print(f" -> Success! Saved to: '{output_filename}'")

        except Exception as e:
            print(f" -> ERROR processing '{base_filename}': {e}")

# --- CLEANUP ---

tif_files = glob.glob(os.path.join(OUTPUT_FOLDER, '*.tif'))

if len(tif_files) > MAX_FILES_TO_KEEP:
    tif_files.sort(key=os.path.getmtime)
    
    num_to_delete = len(tif_files) - MAX_FILES_TO_KEEP
    files_to_delete = tif_files[:num_to_delete]
    
    print(f"Found {len(tif_files)} files. Deleting the {num_to_delete} oldest ones.")
    for file_path in files_to_delete:
        print(f" -> Deleting: {os.path.basename(file_path)}")
        os.remove(file_path)
else:
    print("Cleanup not needed.")

print("\n--- Process Finished ---")

--- Starting Conversion Process ---
Found 12 files to process.

Processing: 'RRQPE-INST-GLB-5_v1r1_blend_s202509290750000_e202509290759599_c202509290811102.nc'
 -> ERROR processing 'RRQPE-INST-GLB-5_v1r1_blend_s202509290750000_e202509290759599_c202509290811102.nc': "'latitude' is not a valid dimension or coordinate for Dataset with dimensions FrozenMappingWarningOnValuesAccess({'Rows': 6501, 'Columns': 18000})"
Processing: 'RRQPE-INST-GLB-5_v1r1_blend_s202509290800000_e202509290809599_c202509290821500.nc'
 -> ERROR processing 'RRQPE-INST-GLB-5_v1r1_blend_s202509290800000_e202509290809599_c202509290821500.nc': "'latitude' is not a valid dimension or coordinate for Dataset with dimensions FrozenMappingWarningOnValuesAccess({'Rows': 6501, 'Columns': 18000})"
Processing: 'RRQPE-INST-GLB-5_v1r1_blend_s202509290810000_e202509290819599_c202509290831212.nc'
 -> ERROR processing 'RRQPE-INST-GLB-5_v1r1_blend_s202509290810000_e202509290819599_c202509290831212.nc': "'latitude' is not a valid dimen